In [1]:
from numpy import linalg as LA
from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score
from multi import *
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# MPPM

## Data Generation

In [2]:
## Parameters
n = 600
k = 2
L = 2
rho = 2*np.log(n)/n
pi = [1/2,1/2]
in_list = [4,4]
out_list = [2,3.5]

## Generate multi-layer adjacency matrices
Ares,gt = genPPM(n,k,L,rho,pi,in_list,out_list)

## Optimal weight
print(lam_norm(in_list,out_list,k=2))

[0.8333333333333334, 0.16666666666666669]


## Algorithms

SCME_gm

In [3]:
test = SCME(Ares,k,method = 'gmm')
label,lamb,eratio = test.multipleUpdate(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.869 0.131]
ARI:  0.9215


SCME_km

In [4]:
test = SCME(Ares,k,method = 'km')
label,lamb,eratio = test.multipleUpdate(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.869 0.131]
ARI:  0.9279


ISC_gm

In [5]:
test = ISC(Ares,k,method = 'gmm')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.9066 0.0934]
ARI:  0.9279


ISC_km

In [6]:
test = ISC(Ares,k,method = 'km')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.9968 0.0032]
ARI:  0.9279


ASC

In [7]:
kmeans = ASC(Ares,k)
print(adjusted_rand_score(kmeans.labels_,gt))

0.33529049861300675


MAM

In [8]:
asso = [True]*L
test = MAM(Ares,asso,k)
_,mam = test.Mam()
print(adjusted_rand_score(SC(mam,k),gt))

0.22938592696118118


Mean adj

In [9]:
print(adjusted_rand_score(SC(sum([a*b for a,b in zip([1/L]*L,Ares)]),k),gt))

0.7506953968800606


# SBM

## Data Generation

In [10]:
## Parameters
n = 600
k = 2
L = 2
rho = 1.6*np.log(n)/n
pi = [1/2,1/2]
Alist = [np.array([[5,2],[2,4]]),np.array([[4,3.5],[3.5,5]])]

## Generate multi-layer adjacency matrices
Ares,gt = genSBM(n,k,L,rho,pi,Alist)

## Algorithms

SCME_gm

In [11]:
test = SCME(Ares,k,method = 'gmm')
label,lamb,eratio = test.multipleUpdate(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.7264 0.2736]
ARI:  0.9279


SCME_km

In [12]:
test = SCME(Ares,k,method = 'km')
label,lamb,eratio = test.multipleUpdate(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.7264 0.2736]
ARI:  0.9473


ISC_gm

In [13]:
test = ISC(Ares,k,method = 'gmm')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.7964 0.2036]
ARI:  0.9151


ISC_km

In [14]:
test = ISC(Ares,k,method = 'km')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.992 0.008]
ARI:  0.8897


ASC

In [15]:
kmeans = ASC(Ares,k)
print(adjusted_rand_score(kmeans.labels_,gt))

0.35893063748015896


MAM

In [16]:
asso = [True]*L
test = MAM(Ares,asso,k)
_,mam = test.Mam()
print(adjusted_rand_score(SC(mam,k),gt))

0.2590937497183385


Mean adj

In [17]:
evals, evecs = largest_eigsh(sum([a*b for a,b in zip([1/L]*L,Ares)]),k+1, which='LM')
evals_abs = abs(evals)
evecs = evecs[:,evals_abs.argsort()]
evals = evals[evals_abs.argsort()]
model = GaussianMixture(n_components=k,n_init=10).fit(np.real(evecs[:,1:]))
label = model.predict(np.real(evecs[:,1:]))
print(adjusted_rand_score(label,gt))

0.889694320656579
